In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras import preprocessing

# Utilizaremos las 10,000 palabras más usuales.
max_features = 10000

# Por cada crítica, consideraremos sólo las 20 primeras palabras dentro de las 10,000 más usuales.
maxlen = 20

# Cargamos los datos como listas de enteros.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# Convertimos nuestras listas de enteros en tensores 2D de dimensiones (número de críticas, maxlen)
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

17465344/17464789 [==============================] - 0s 0us/step


In [ ]:
x_train.shape

(25000, 20)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

model = Sequential()
# Definimos las características de la capa Embedding.
# El primer argumento se refiere al tamaño del vocabulario.
# El segundo argumento está asociado con las dimensiones que tendrá cada word embedding
# Y el tercer argumento, nos va a servir para aplanar los tensores que se vayan generando.
model.add(Embedding(10000, 8, input_length=maxlen))
# Luego de la capa Embedding, las dimensiones de las activaciones serán (tamaño del vocabulario, maxlen, 8)

# Procedemos a aplanar el tensor 3D de word embeddings a un tensor 2D con dimensiones (tamaño del vocabulario, maxlen * 8)
model.add(Flatten())

# Agregamos el clasificador
model.add(Dense(1, activation='sigmoid'))
#Compilación
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
#Resumen de la arquitectura
model.summary()

#Definimos los callbacks
es = EarlyStopping(monitor= 'val_loss', mode='min', patience = 7, verbose=1)
mc = ModelCheckpoint('/content/model_embedding_imdb.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)


#Entrenamiento
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 8)             80000     
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 1s 2ms/step - loss: 0.6738 - acc: 0.6118 - val_loss: 0.6306 - val_acc: 0.6900
Epoch 2/10
625/625 [==============================] - 1s 2ms/step - loss: 0.5520 - acc: 0.7485 - val_loss: 0.5319 - val_acc: 0.7264
Epoch 3/10
625/625 [==============================] - 1s 1ms/step - loss: 0.4651 - acc: 0.7869 - val_loss: 0.5046 - 

In [ ]:
!cp '/content/model_embedding_imdb.h5'  '/content/drive/My Drive/Natural_Language_Processing/Models/model_embedding_imdb.h5'